In [5]:
import numpy as np
import cv2
import os
from tqdm import tqdm
from skimage.feature import hog, local_binary_pattern
from sklearn.decomposition import PCA

DATASET_PATH = "E:/Coding/Advanced ML/sample_train_data"
IMG_SIZE = (64, 64)

In [ ]:
def process_images(extract_func, desc="Processing"):
    images = []
    filenames = os.listdir(DATASET_PATH)
    
    for file in tqdm(filenames, desc=desc):  
        img_path = os.path.join(DATASET_PATH, file)
        img = cv2.imread(img_path)

        if img is None:
            continue  # Skip unreadable images
        
        img = cv2.resize(img, IMG_SIZE)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        
        features = extract_func(img, gray)  # Extract features
        images.append(features)
    
    return np.array(images)

# RGBW

Image Processing

In [7]:
def extract_rgbw_features(img):
    r, g, b = cv2.split(img)
    w = (r + g + b) // 3  
    return np.array([r.mean(), g.mean(), b.mean(), w.mean()])

In [8]:
CACHE_FILE = "features_rgbw.npy"
rgbw_features = process_images(lambda img, gray: extract_rgbw_features(img), "Extracting RGBW")
np.save(CACHE_FILE, rgbw_features)

Extracting RGBW: 100%|██████████| 20000/20000 [03:11<00:00, 104.55it/s]


# HOG

Image Processing

In [11]:
def extract_hog_features(gray):
    features, _ = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    hist, _ = np.histogram(features, bins=32, range=(0, np.max(features)), density=True)
    return hist

In [12]:
CACHE_FILE = "features_hog.npy"
hog_features = process_images(lambda img, gray: extract_hog_features(gray), "Extracting HOG")
np.save(CACHE_FILE, hog_features)

Extracting HOG: 100%|██████████| 20000/20000 [08:09<00:00, 40.85it/s]


# LBP

Image Processing

In [13]:
def extract_lbp_features(gray):
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=59, range=(0, 59), density=True)
    return hist

In [14]:
CACHE_FILE = "features_lbp.npy"
lbp_features = process_images(lambda img, gray: extract_lbp_features(gray), "Extracting LBP")
np.save(CACHE_FILE, lbp_features)

Extracting LBP: 100%|██████████| 20000/20000 [01:03<00:00, 317.23it/s]


# Fourier Transform

Image Processing

In [15]:
def extract_fourier_features(gray):
    H, W = gray.shape
    fft_img = np.fft.fft2(gray)
    fft_shift = np.fft.fftshift(fft_img)
    magnitude_spectrum = np.abs(fft_shift)
    low_freq = magnitude_spectrum[:H//4, :W//4].mean()
    mid_freq = magnitude_spectrum[H//4:H//2, W//4:W//2].mean()
    high_freq = magnitude_spectrum[H//2:, W//2:].mean()
    return np.array([low_freq, mid_freq, high_freq])

In [16]:
CACHE_FILE = "features_fda.npy"
fourier_features = process_images(lambda img, gray: extract_fourier_features(gray), "Extracting Fourier")
np.save(CACHE_FILE, fourier_features)

Extracting Fourier: 100%|██████████| 20000/20000 [00:51<00:00, 387.33it/s]


# Error Level Analysis

Image Processing

In [17]:
def extract_ela_features(img):
    cv2.imwrite("temp.jpg", img, [cv2.IMWRITE_JPEG_QUALITY, 90])
    compressed = cv2.imread("temp.jpg")
    ela = cv2.absdiff(img, compressed)
    return np.array([ela.mean()])

In [18]:
CACHE_FILE = "features_ela.npy"
ela_features = process_images(lambda img, gray: extract_ela_features(img), "Extracting ELA")
np.save(CACHE_FILE, ela_features)

Extracting ELA: 100%|██████████| 20000/20000 [02:53<00:00, 115.13it/s]


# Local Binary Pattern Variance

Image Processing

In [25]:
def extract_lbpv_features(gray):
    if gray is None or gray.size == 0:
        raise ValueError("Invalid image: empty or not loaded correctly")

    lbpv = local_binary_pattern(gray, P=8, R=1, method="var")

    # Replace NaN and Inf values
    lbpv = np.nan_to_num(lbpv, nan=0)

    # Ensure max is finite
    max_val = np.max(lbpv)
    if not np.isfinite(max_val) or max_val == 0:
        max_val = 1  # Avoid division issues

    hist, _ = np.histogram(lbpv.ravel(), bins=59, range=(0, max_val), density=True)
    return hist

In [26]:
CACHE_FILE = "features_lbpv.npy"
lbpv_features = process_images(lambda img, gray: extract_lbpv_features(gray), "Extracting LBPV")
np.save(CACHE_FILE, lbpv_features)

Extracting LBPV: 100%|██████████| 20000/20000 [01:05<00:00, 304.18it/s]


# Color Gamut

Image Processing

In [27]:
def extract_color_gamut(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return hist.flatten()

In [28]:
CACHE_FILE = "features_colorgamut.npy"
color_gamut_features = process_images(lambda img, gray: extract_color_gamut(img), "Extracting Color Gamut")
np.save(CACHE_FILE, color_gamut_features)

Extracting Color Gamut: 100%|██████████| 20000/20000 [00:46<00:00, 425.80it/s]


# Fractal Dimension

Image Processing

In [31]:
def fractal_dimension(gray, threshold=128):  
    # Ensure the image is binary by thresholding
    img = gray < threshold  
    sizes = 2 ** np.arange(1, 6)

    counts = []
    for size in sizes:
        S = np.floor(gray.shape[0] / size) * size  # Ensure divisibility
        box_count = np.sum(img[:int(S), :int(S)].reshape(-1, size, size), axis=(1, 2))
        counts.append(np.sum(box_count > 0))

    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)  
    return np.array([coeffs[0]])  # Return as a feature array


In [32]:
CACHE_FILE = "features_fractal.npy"
fractal_features = process_images(lambda img, gray: fractal_dimension(gray), "Extracting Fractal Dimension")
np.save(CACHE_FILE, fractal_features)

Extracting Fractal Dimension:  20%|█▉        | 3973/20000 [00:10<00:41, 390.44it/s]C:\Users\user\AppData\Local\Temp\ipykernel_16936\2295806612.py:12: RuntimeWarning: divide by zero encountered in log
  coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
Extracting Fractal Dimension: 100%|██████████| 20000/20000 [00:50<00:00, 394.24it/s]
